# IMDB Review Classifier

## Imports for Data cleaning

In [1]:
from __future__ import division, print_function
%matplotlib inline
from importlib import reload  # Python 3
import utils; reload(utils)
from utils import *

Using TensorFlow backend.


In [2]:
path = "imdb/"
model_path = path + 'models/'
import os
if not os.path.exists(model_path): os.mkdir(model_path)

In [17]:
from keras.utils.data_utils import get_file
import pickle
from keras.preprocessing import sequence
import numpy as np

In [8]:
from keras.datasets import imdb
idx = imdb.get_word_index()

In [9]:
idx_arr = sorted(idx, key=idx.get)
idx_arr[:10]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i']

In [10]:
idx2word = {v: k for k, v in idx.items()}

In [11]:
path = get_file('imdb_full.pkl',
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')
f = open(path, 'rb')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

In [12]:
len(x_train)

25000

In [13]:
', '.join(map(str, x_train[0]))

'23022, 309, 6, 3, 1069, 209, 9, 2175, 30, 1, 169, 55, 14, 46, 82, 5869, 41, 393, 110, 138, 14, 5359, 58, 4477, 150, 8, 1, 5032, 5948, 482, 69, 5, 261, 12, 23022, 73935, 2003, 6, 73, 2436, 5, 632, 71, 6, 5359, 1, 25279, 5, 2004, 10471, 1, 5941, 1534, 34, 67, 64, 205, 140, 65, 1232, 63526, 21145, 1, 49265, 4, 1, 223, 901, 29, 3024, 69, 4, 1, 5863, 10, 694, 2, 65, 1534, 51, 10, 216, 1, 387, 8, 60, 3, 1472, 3724, 802, 5, 3521, 177, 1, 393, 10, 1238, 14030, 30, 309, 3, 353, 344, 2989, 143, 130, 5, 7804, 28, 4, 126, 5359, 1472, 2375, 5, 23022, 309, 10, 532, 12, 108, 1470, 4, 58, 556, 101, 12, 23022, 309, 6, 227, 4187, 48, 3, 2237, 12, 9, 215'

In [14]:
idx2word[23022]

'bromwell'

In [15]:
' '.join([idx2word[o] for o in x_train[0]])

"bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i'm here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn't"

In [16]:
labels_train[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [18]:
vocab_size = 5000

trn = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_train]
test = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_test]
# check the "max index" word out of curiosity
idx2word[vocab_size-1]

'bergman'

In [19]:
lens = np.array([*map(len, trn)])
(lens.max(), lens.min(), lens.mean())

(2493, 10, 237.71364)

## Padding the cells to get a same length sequence

In [20]:
seq_len = 500

trn = sequence.pad_sequences(trn, maxlen=seq_len, value=0)
test = sequence.pad_sequences(test, maxlen=seq_len, value=0)

In [21]:
trn.shape

(25000, 500)

In [22]:
batch_size = 64

## Creating a LSTM based model

In [23]:
from keras.models import Sequential
from keras.layers import Dense, Activation

In [24]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len, mask_zero=True,
              embeddings_regularizer=l2(1e-6)),
    SpatialDropout1D(0.2),
    LSTM(100, implementation=2),
    Dense(1, activation='sigmoid')])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 500, 32)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.fit(trn, labels_train, validation_data=(test, labels_test), epochs=5, batch_size=batch_size)

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 464s 19ms/step - loss: 0.4707 - acc: 0.7677 - val_loss: 0.3368 - val_acc: 0.8657
Epoch 2/5
25000/25000 [==============================] - 469s 19ms/step - loss: 0.3055 - acc: 0.8787 - val_loss: 0.3682 - val_acc: 0.8409
Epoch 3/5
25000/25000 [==============================] - 496s 20ms/step - loss: 0.2580 - acc: 0.9018 - val_loss: 0.3176 - val_acc: 0.8686
Epoch 4/5
25000/25000 [==============================] - 502s 20ms/step - loss: 0.2308 - acc: 0.9132 - val_loss: 0.3120 - val_acc: 0.8768
Epoch 5/5
25000/25000 [==============================] - 517s 21ms/step - loss: 0.2247 - acc: 0.9137 - val_loss: 0.3714 - val_acc: 0.8623
